In [3]:
import numpy as np

In [10]:
AGENT_STATES = 3*3
TARGET_STATES = 3*3
CALL_STATES = 2

AGENT_ACTIONS = 5
TARGET_ACTIONS = 5
CALL_ACTIONS = 2 # (IF ON->(ON, OFF)  OFF->(OFF, ON))

AGENT_STATE_VALUES = tuple(range(AGENT_STATES))
TARGET_STATE_VALUES = tuple(range(TARGET_STATES))
CALL_STATE_VALUES = tuple(range(CALL_STATES))

AGENT_ACTION_VALUES = 5
TARGET_ACTION_VALUES = 5
CALL_ACTIONS_VALUES = 2

NUM_STATES = AGENT_STATES*TARGET_STATES*CALL_STATES
NUM_ACTIONS = AGENT_ACTIONS*TARGET_ACTIONS*CALL_ACTIONS

ACTIONS = ['UP', 'RIGHT', 'DOWN', 'LEFT', 'STAY']
TARGET_ACTION_PROB = [0.15, 0.15, 0.15, 0.15, 0.40]

X = 0.89

In [9]:
class State:
    def __init__(self, agent, target, call):
        if (agent not in AGENT_STATE_VALUES) or (target not in TARGET_STATE_VALUES) or (call not in CALL_STATE_VALUES):
            raise ValueError
        self.agent = agent
        self.target = target
        self.call = call
    
    def asTuple(self):
        return (self.agent, self.target, self.call)
    
    def asList(self):
        return [self.agent, self.target, self.call]
    
    def getHash(self):
        return (self.agent * TARGET_STATES * CALL_STATES +
                self.target * CALL_STATES +
                self.call)
    @classmethod
    def fromHash(cls, num):
        if type(num)!= int:
            raise ValueError
        if num not in (0,NUM_STATES):
            raise ValueError
        
        agent = num // (TARGET_STATES*CALL_STATES)
        num = num % (TARGET_STATES*CALL_STATES)
        
        target = num / CALL_STATES
        num = num % CALL_STATES
        
        call = num
        return State(agent, target, call)
            

In [11]:
class Action:
    def __init__(self, agent, target, call):
        if (agent not in AGENT_ACTION_VALUES) or (target not in TARGET_ACTION_VALUES) or (call not in CALL_ACTION_VALUES):
            raise ValueError
        self.agent = agent
        self.target = target
        self.call = call
    
    def asTuple(self):
        return (self.agent, self.target, self.call)
    
    def asList(self):
        return [self.agent, self.target, self.call]
    
    def getHash(self):
        return (self.agent * TARGET_ACTIONS * CALL_ACTIONS +
                self.target * CALL_ACTIONS +
                self.call)
    @classmethod
    def fromHash(cls, num):
        if type(num)!= int:
            raise ValueError
        if num not in (0,NUM_STATES):
            raise ValueError
        
        agent = num // (TARGET_ACTIONS*CALL_ACTIONS)
        num = num % (TARGET_ACTIONS*CALL_ACTIONS)
        
        target = num / CALL_ACTIONS
        num = num % CALL_ACTIONS
        
        call = num
        return Action(agent, target, call)